## Part 1 - Data Preparation and Exploration 

In [1]:
%%capture
# Due to the configuration of the base Jupter image, the following imports are required for the regressions in the assignment to report the correct metrics

import sys 
!{sys.executable} -m pip uninstall statsmodels --yes 
!{sys.executable} -m pip uninstall numpy --yes
!{sys.executable} -m pip uninstall pandas --yes 
!{sys.executable} -m pip uninstall patsy --yes 
!{sys.executable} -m pip install numpy==1.17
!{sys.executable} -m pip install pandas==1.0
!{sys.executable} -m pip install patsy==0.5.2
!{sys.executable} -m pip install statsmodels==0.11.1

In [2]:
#Import Libraries

import pandas as pd
import datetime as dt
import scipy.stats as sp
import numpy as np
import statsmodels.formula.api as sm 

In [3]:
# Import Shotlog_14_15 and Player_Stats Datasets

Shotlog_1415=pd.read_csv("Assignment Data/Week 6/Shotlog_14_15.csv")
Player_Stats=pd.read_csv("Assignment Data/Week 6/Player_Stats_14_15.csv")
display(Shotlog_1415)

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,...,closest_defender,closest_defender_id,closest_def_dist,current_shot_hit,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game
0,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,1,1,...,"Lopez, Brook",201572,6.6,1,2,al horford,201143,0.541259,715,10
1,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,2,1,...,"Lopez, Brook",201572,5.6,0,0,al horford,201143,0.541259,715,10
2,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,3,1,...,"Lopez, Brook",201572,4.7,0,0,al horford,201143,0.541259,715,10
3,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,4,1,...,"Lopez, Brook",201572,5.8,0,0,al horford,201143,0.541259,715,10
4,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,5,2,...,"Lopez, Brook",201572,6.4,0,0,al horford,201143,0.541259,715,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128064,21400350,14-Dec-14,WAS vs. UTA,WAS,UTA,H,W,9,11,3,...,"Burke, Trey",203504,4.7,1,2,john wall,202322,0.448513,874,15
128065,21400350,14-Dec-14,WAS vs. UTA,WAS,UTA,H,W,9,12,3,...,"Exum, Dante",203957,3.4,1,2,john wall,202322,0.448513,874,15
128066,21400350,14-Dec-14,WAS vs. UTA,WAS,UTA,H,W,9,13,4,...,"Kanter, Enes",202683,1.2,0,0,john wall,202322,0.448513,874,15
128067,21400350,14-Dec-14,WAS vs. UTA,WAS,UTA,H,W,9,14,4,...,"Kanter, Enes",202683,1.4,1,2,john wall,202322,0.448513,874,15


In [4]:
Shotlog_1415.columns

Index(['game_id', 'date', 'match', 'home_team', 'away_team', 'home_away',
       'result', 'final_margin', 'shot_number', 'quarter', 'game_clock',
       'shot_clock', 'dribbles', 'touch_time', 'shot_dist', 'points',
       'current_shot_outcome', 'closest_defender', 'closest_defender_id',
       'closest_def_dist', 'current_shot_hit', 'points_earned', 'shoot_player',
       'player_id', 'average_hit', 'shot_count', 'shot_per_game'],
      dtype='object')

In [5]:
Shotlog_1415.groupby("shoot_player").count()

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,...,current_shot_outcome,closest_defender,closest_defender_id,closest_def_dist,current_shot_hit,points_earned,player_id,average_hit,shot_count,shot_per_game
shoot_player,,,,,,,,,,,,,,,,,,,,,
aaron brooks,561,561,561,561,561,561,561,561,561,561,...,561,561,561,561,561,561,561,561,561,561
aaron gordon,104,104,104,104,104,104,104,104,104,104,...,104,104,104,104,104,104,104,104,104,104
al farouq aminu,258,258,258,258,258,258,258,258,258,258,...,258,258,258,258,258,258,258,258,258,258
al horford,715,715,715,715,715,715,715,715,715,715,...,715,715,715,715,715,715,715,715,715,715
al jefferson,800,800,800,800,800,800,800,800,800,800,...,800,800,800,800,800,800,800,800,800,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wesley matthews,748,748,748,748,748,748,748,748,748,748,...,748,748,748,748,748,748,748,748,748,748
wilson chandler,733,733,733,733,733,733,733,733,733,733,...,733,733,733,733,733,733,733,733,733,733
zach lavine,371,371,371,371,371,371,371,371,371,371,...,371,371,371,371,371,371,371,371,371,371


In [6]:
Shotlog_1415["shot_clock"].isna().sum()

5567

In [7]:
Shotlog_1415.groupby("game_id").count()

,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,game_clock,...,closest_defender,closest_defender_id,closest_def_dist,current_shot_hit,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game
game_id,,,,,,,,,,,,,,,,,,,,,
21400001,162,162,162,162,162,162,162,162,162,162,...,162,162,162,162,162,162,162,162,162,162
21400002,137,137,137,137,137,137,137,137,137,137,...,137,137,137,137,137,137,137,137,137,137
21400003,117,117,117,117,117,117,117,117,117,117,...,117,117,117,117,117,117,117,117,117,117
21400004,161,161,161,161,161,161,161,161,161,161,...,161,161,161,161,161,161,161,161,161,161
21400005,111,111,111,111,111,111,111,111,111,111,...,111,111,111,111,111,111,111,111,111,111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21400904,124,124,124,124,124,124,124,124,124,124,...,124,124,124,124,124,124,124,124,124,124
21400905,128,128,128,128,128,128,128,128,128,128,...,128,128,128,128,128,128,128,128,128,128
21400906,148,148,148,148,148,148,148,148,148,148,...,148,148,148,148,148,148,148,148,148,148


In [8]:
Player_Stats.head()

,shoot_player,average_hit
0,aaron brooks,0.415330
1,aaron gordon,0.528846
2,al farouq aminu,0.430233
3,al horford,0.541259
4,al jefferson,0.477500


Convert the “date” variable to a date type variable and calculate summary statistics for the “shot_clock” variable.

In [9]:
Shotlog_1415["date"] = pd.to_datetime(Shotlog_1415["date"])

Create a lagged variable “lag_shot_hit” to indicate the result of the previous shot by the same player at the same game.
Hint: In this dataset, the variable “match” may not be able to uniquely identify each game; you can use “game_id” instead.
You can sort the data by shot number for each player to create the lagged variable. 

In [10]:
Shotlog_1415["lag_shot_hit"] = Shotlog_1415.groupby(["date","shoot_player"])["current_shot_hit"].shift(1)

Create a variable “error” to indicate the prediction error for each shot and a variable “lagerror” for the prediction error for the previous shot. The “error” variable is defined as the difference between the outcome of the current shot and the average success rate (“average_hit”) and the “lagerror” variable is defined as the difference between the outcome of the previous shot and the average success rate.

In [11]:
Shotlog_1415["error"] = Shotlog_1415["current_shot_hit"] - Shotlog_1415["average_hit"]

In [12]:
Shotlog_1415["lagerror"] = Shotlog_1415["lag_shot_hit"] - Shotlog_1415["average_hit"]

In [13]:
Shotlog_1415[["error","lagerror"]].describe()

,error,lagerror
count,1.280690e+05,113726.000000
mean,-5.770049e-18,0.006303
std,4.949640e-01,0.496035
min,-7.124682e-01,-0.712468
25%,-4.491979e-01,-0.449198
50%,-3.850837e-01,-0.382143
75%,5.395973e-01,0.542254
max,6.914894e-01,0.691489


## Part 2 - Conditional Probability and Autocorrelation

Create a dummy variable “conse_shot” that indicates a player made consecutive shots. 

In [14]:
Shotlog_1415["conse_shot_hit"] = np.where((Shotlog_1415["current_shot_hit"]==1)&(Shotlog_1415["lag_shot_hit"]==1), 1, 0) 
Shotlog_1415.head()

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,...,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game,lag_shot_hit,error,lagerror,conse_shot_hit
0,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,1,1,...,2,al horford,201143,0.541259,715,10,NaN,0.458741,NaN,0
1,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,2,1,...,0,al horford,201143,0.541259,715,10,1.0,-0.541259,0.458741,0
2,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,3,1,...,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259,0
3,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,4,1,...,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259,0
4,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,5,2,...,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259,0


Create a dataframe “Player_Prob” for the probability of making the previous shot and the joint probability for making both the previous and current shots. Name the probability of making the previous shot “average_lag_hit” and the probability of making both shots “conse_shot_hit.” 

In [15]:
Player_Prob = pd.DataFrame()
Player_Prob["average_lag_hit"] = Shotlog_1415.groupby("shoot_player")["lag_shot_hit"].mean()
Player_Prob["conse_shot_hit"] = Shotlog_1415.groupby("shoot_player")["conse_shot_hit"].mean()

Player_Prob.head()

,average_lag_hit,conse_shot_hit
shoot_player,,
aaron brooks,0.418000,0.153298
aaron gordon,0.532468,0.201923
al farouq aminu,0.465686,0.162791
al horford,0.537994,0.262937
al jefferson,0.480000,0.207500


In the “Player_Prob” dataframe, calculate the conditional probability “conditional_prob” for a player to make a shot given that he made the previous shot. 

In [16]:
Player_Prob["conditional_prob"] = Player_Prob["conse_shot_hit"]/Player_Prob["average_lag_hit"]

Merge the “Player_Prob” dataframe into the “Player_Stats” dataframe.

In [17]:
Player_Stats = pd.merge( Player_Stats, Player_Prob, on = ["shoot_player"])
Player_Stats.head()

,shoot_player,average_hit,average_lag_hit,conse_shot_hit,conditional_prob
0,aaron brooks,0.415330,0.418000,0.153298,0.366741
1,aaron gordon,0.528846,0.532468,0.201923,0.379221
2,al farouq aminu,0.430233,0.465686,0.162791,0.349572
3,al horford,0.541259,0.537994,0.262937,0.488736
4,al jefferson,0.477500,0.480000,0.207500,0.432292


Calculate summary statistics for the probability for a player to make a shot (“average_hit”) and the conditional probability for a player to make a shot given that he made the previous one (“conditional_prob”) and the probability of players making consecutive shots (“conse_shot_hit”).

In [18]:
Player_Stats[["average_hit", "conditional_prob", "conse_shot_hit"]].describe()

,average_hit,conditional_prob,conse_shot_hit
count,281.000000,281.000000,281.000000
mean,0.451545,0.380233,0.176987
std,0.059392,0.062320,0.047943
min,0.308511,0.225801,0.076190
25%,0.413223,0.336689,0.144543
50%,0.446078,0.381570,0.171625
75%,0.480480,0.422801,0.203512
max,0.712468,0.613209,0.422392


Perform a t-test for the statistical significance on the difference between conditional probability and unconditional probability of making a shot.

In [19]:
sp.stats.ttest_ind(Player_Stats['conditional_prob'], Player_Stats['average_hit'])

Ttest_indResult(statistic=-13.885932802814914, pvalue=6.925846314604593e-38)

Calculate the first order autocorrelation coefficient on making a shot (correlation coefficient between making the current shot and the previous shot) for the entire shotlog dataset.

In [20]:
Shotlog_1415["current_shot_hit"].corr(Shotlog_1415["lag_shot_hit"])

-0.010502388301693177

Calculate the first order autocorrelation coefficient on making a shot for each player. Display the top ten players with the highest first order autocorrelation coefficient.

In [21]:
auto_corr_by_player = Shotlog_1415.groupby("shoot_player")[["current_shot_hit", "lag_shot_hit"]].corr().unstack().iloc[:, 1].reset_index()
auto_corr_by_player.columns = ["shoot_player", "autocorr"]
auto_corr_by_player.sort_values( by = ["autocorr"], ascending = False ).head(10)

,shoot_player,autocorr
131,joey dorsey,0.334252
54,cole aldrich,0.174666
200,nate robinson,0.122107
267,tyler hansbrough,0.120608
7,alex len,0.118461
50,cj mccollum,0.115949
114,jason smith,0.105903
190,matt bonner,0.098577
143,jusuf nurkic,0.097465
195,mike miller,0.089366


## Part 3 - Regression Analyses

Reg1: Run a linear least squares regression using the entire shotlog dataframe. Include the following control variables:

Shot distance

Number of dribbles

Touch time

Type of shot (“points” variable)

Quarter of the game (as a categorical variable)

Home or away game

Shoot_player

Closest defender

Closest defender distance

In [52]:
'''
Index(['game_id', 'date', 'match', 'home_team', 'away_team', 'home_away',
       'result', 'final_margin', 'shot_number', 'quarter', 'game_clock',
       'shot_clock', 'dribbles', 'touch_time', 'shot_dist', 'points',
       'current_shot_outcome', 'closest_defender', 'closest_defender_id',
       'closest_def_dist', 'current_shot_hit', 'points_earned', 'shoot_player',
       'player_id', 'average_hit', 'shot_count', 'shot_per_game'],
      dtype='object')
'''

"\nIndex(['game_id', 'date', 'match', 'home_team', 'away_team', 'home_away',\n       'result', 'final_margin', 'shot_number', 'quarter', 'game_clock',\n       'shot_clock', 'dribbles', 'touch_time', 'shot_dist', 'points',\n       'current_shot_outcome', 'closest_defender', 'closest_defender_id',\n       'closest_def_dist', 'current_shot_hit', 'points_earned', 'shoot_player',\n       'player_id', 'average_hit', 'shot_count', 'shot_per_game'],\n      dtype='object')\n"

In [23]:
formula = "error ~ lagerror + shot_dist + dribbles + touch_time + C(points) + C(quarter) + home_away + C(shoot_player) + C(closest_defender) + closest_def_dist"
reg1 = sm.ols(formula = formula, data = Shotlog_1415).fit()
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  error   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     9.002
Date:                Sun, 10 Mar 2024   Prob (F-statistic):               0.00
Time:                        04:38:45   Log-Likelihood:                -78065.
No. Observations:              113726   AIC:                         1.577e+05
Df Residuals:                  112960   BIC:                         1.650e+05
Df Model:                         765                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            0.1712      0.039      4.446      0.000       0.096       0.247
C(points)[T.3]                                       0.0362      0.005      6.865      0.000       0.026       0.047
C(quarter)[T.2]                                     -0.0114      0.004     -2.600      0.009      -0.020      -0.003
C(quarter)[T.3]                                     -0.0023      0.004     -0.543      0.587      -0.011       0.006
C(quarter)[T.4]                                     -0.0118      0.004     -2.661      0.008      -0.021      -0.003
C(quarter)[T.5]                                     -0.0446      0.016     -2.713      0.007      -0.077      -0.012
C(quarter)[T.6]                                      0.0078      0.038      0.207      0.836      -0.066       0.081
C(quarter)[T.7]                                     -0.0191      0.074     -0.258      0.796      -0.164       0.126
home_away[T.H]                                       0.0066      0.003      2.297      0.022       0.001       0.012
C(shoot_player)[T.aaron gordon]                     -0.0798      0.059     -1.344      0.179      -0.196       0.037
C(shoot_player)[T.al farouq aminu]                  -0.0823      0.040     -2.039      0.041      -0.161      -0.003
C(shoot_player)[T.al horford]                       -0.0605      0.029     -2.088      0.037      -0.117      -0.004
C(shoot_player)[T.al jefferson]                     -0.0286      0.028     -1.009      0.313      -0.084       0.027
C(shoot_player)[T.alan anderson]                    -0.0123      0.036     -0.339      0.735      -0.083       0.059
C(shoot_player)[T.alan crabbe]                      -0.0599      0.065     -0.920      0.357      -0.187       0.068
C(shoot_player)[T.alex len]                         -0.1226      0.038     -3.215      0.001      -0.197      -0.048
C(shoot_player)[T.alexis ajinca]                    -0.1059      0.044     -2.432      0.015      -0.191      -0.021
C(shoot_player)[T.alonzo gee]                       -0.0585      0.053     -1.107      0.268      -0.162       0.045
C(shoot_player)[T.amare stoudemire]                 -0.0813      0.036     -2.273      0.023      -0.151      -0.011
C(shoot_player)[T.amir johnson]                     -0.1183      0.034     -3.469      0.001      -0.185      -0.051
C(shoot_player)[T.andre drummond]                   -0.1371      0.030     -4.641      0.000      -0.195      -0.079
C(shoot_player)[T.andre iguodala]                   -0.0302      0.036     -0.840      0.401      -0.101       0.040
C(shoot_player)[T.andre miller]                     -0.0572      0.049     -1.172      0.241      -0.153       0.038
C(shoot_player)[T.andre roberson]        

Reg2: Run a weighted least squares regression using the entire shotlog dataframe. Include the same set of control variables as in Reg1. The regression should be weighted by the number of shot per game (weight=1/shot_per_game)

In [25]:
formula = "error ~ lagerror + shot_dist + dribbles + touch_time + C(points) + C(quarter) + home_away + C(shoot_player) + C(closest_defender) + closest_def_dist"
reg2 = sm.wls(formula = formula, weights = 1/Shotlog_1415['shot_per_game'], data = Shotlog_1415).fit()
reg2.summary()

                            WLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.062
Model:                            WLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     9.738
Date:                Sun, 10 Mar 2024   Prob (F-statistic):               0.00
Time:                        05:12:06   Log-Likelihood:                -86948.
No. Observations:              113726   AIC:                         1.754e+05
Df Residuals:                  112960   BIC:                         1.828e+05
Df Model:                         765                                         
Covariance Type:            nonrobust                                         
                                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------

Reg3_player: Run linear least squares regressions on individual players. Include the following control variables:

Shot distance

Number of dribbles

Touch time

Type of shot (“points” variable)

Quarter of the game (as a categorical variable)

Home or away game

Closest defender distance

In [37]:
formula = "error ~ lagerror + shot_dist + dribbles + touch_time + C(points) + C(quarter) + home_away + closest_def_dist"
results_3 = {}


for player in Shotlog_1415['shoot_player'].unique():
    player_data = Shotlog_1415[Shotlog_1415['shoot_player'] == player]

    reg3 = sm.ols(formula=formula, data=player_data ).fit()
    results_3[player] = reg3

In [41]:
results_3["james harden"].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  error   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     4.952
Date:                Sun, 10 Mar 2024   Prob (F-statistic):           1.61e-07
Time:                        05:59:37   Log-Likelihood:                -691.15
No. Observations:                 995   AIC:                             1406.
Df Residuals:                     983   BIC:                             1465.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.1003      0.053      1.902      0.058      -0.003       0.204
C(points)[T.3]       0.0715      0.061      1.179      0.239      -0.048       0.191
C(quarter)[T.2]      0.0061      0.045      0.134      0.894      -0.083       0.095
C(quarter)[T.3]     -0.0207      0.042     -0.494      0.622      -0.103       0.062
C(quarter)[T.4]      0.0091      0.048      0.192      0.848      -0.084       0.102
C(quarter)[T.5]      0.1730      0.159      1.085      0.278      -0.140       0.486
home_away[T.H]       0.0023      0.032      0.070      0.944      -0.060       0.065
lagerror             0.0257      0.031      0.821      0.412      -0.036       0.087
shot_dist           -0.0154      0.003     -4.754      0.000      -0.022      -0.009
dribbles             0.0103      0.008      1.224      0.221      -0.006       0.027
touch_time          -0.0127      0.010     -1.294      0.196      -0.032       0.007
closest_def_dist     0.0341      0.008      4.016      0.000       0.017       0.051
==============================================================================
Omnibus:                     4449.293   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              136.371
Skew:                           0.188   Prob(JB):                     2.44e-30
Kurtosis:                       1.226   Cond. No.                         201.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [42]:
results_3["russell westbrook"].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  error   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     5.529
Date:                Sun, 10 Mar 2024   Prob (F-statistic):           1.32e-08
Time:                        05:59:40   Log-Likelihood:                -631.81
No. Observations:                 923   AIC:                             1288.
Df Residuals:                     911   BIC:                             1346.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0958      0.051      1.893      0.059      -0.004       0.195
C(points)[T.3]      -0.0138      0.060     -0.232      0.817      -0.131       0.103
C(quarter)[T.2]     -0.0034      0.046     -0.075      0.940      -0.093       0.086
C(quarter)[T.3]     -0.0061      0.043     -0.141      0.888      -0.091       0.078
C(quarter)[T.4]      0.0114      0.047      0.241      0.810      -0.081       0.104
C(quarter)[T.5]      0.1294      0.113      1.145      0.253      -0.093       0.351
home_away[T.H]       0.0319      0.032      0.984      0.325      -0.032       0.095
lagerror            -0.0090      0.033     -0.275      0.783      -0.073       0.055
shot_dist           -0.0148      0.003     -5.353      0.000      -0.020      -0.009
dribbles            -0.0119      0.011     -1.121      0.263      -0.033       0.009
touch_time           0.0120      0.013      0.959      0.338      -0.013       0.037
closest_def_dist     0.0207      0.006      3.449      0.001       0.009       0.032
==============================================================================
Omnibus:                     4411.123   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              124.183
Skew:                           0.259   Prob(JB):                     1.08e-27
Kurtosis:                       1.279   Cond. No.                         123.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
results_3["andrew wiggins"].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  error   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     5.602
Date:                Sun, 10 Mar 2024   Prob (F-statistic):           1.14e-08
Time:                        05:59:45   Log-Likelihood:                -500.04
No. Observations:                 738   AIC:                             1024.
Df Residuals:                     726   BIC:                             1079.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.2003      0.057      3.511      0.000       0.088       0.312
C(points)[T.3]       0.0726      0.069      1.053      0.293      -0.063       0.208
C(quarter)[T.2]      0.0087      0.051      0.171      0.865      -0.092       0.109
C(quarter)[T.3]      0.0029      0.048      0.061      0.951      -0.091       0.096
C(quarter)[T.4]      0.0320      0.051      0.624      0.533      -0.069       0.133
C(quarter)[T.5]     -0.1656      0.281     -0.589      0.556      -0.718       0.386
home_away[T.H]       0.0233      0.036      0.647      0.518      -0.047       0.094
lagerror            -0.0016      0.036     -0.044      0.965      -0.072       0.069
shot_dist           -0.0212      0.003     -6.769      0.000      -0.027      -0.015
dribbles            -0.0095      0.017     -0.546      0.585      -0.044       0.025
touch_time          -0.0129      0.018     -0.733      0.464      -0.047       0.022
closest_def_dist     0.0206      0.009      2.384      0.017       0.004       0.038
==============================================================================
Omnibus:                     4176.254   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               89.399
Skew:                           0.260   Prob(JB):                     3.87e-20
Kurtosis:                       1.376   Cond. No.                         241.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [44]:
results_3["stephen curry"].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  error   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     6.498
Date:                Sun, 10 Mar 2024   Prob (F-statistic):           1.79e-10
Time:                        06:00:03   Log-Likelihood:                -625.19
No. Observations:                 910   AIC:                             1274.
Df Residuals:                     898   BIC:                             1332.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.2196      0.057      3.867      0.000       0.108       0.331
C(points)[T.3]       0.1007      0.055      1.818      0.069      -0.008       0.210
C(quarter)[T.2]     -0.0474      0.048     -0.997      0.319      -0.141       0.046
C(quarter)[T.3]     -0.0445      0.042     -1.064      0.288      -0.126       0.038
C(quarter)[T.4]     -0.0776      0.047     -1.641      0.101      -0.170       0.015
C(quarter)[T.5]      0.0963      0.202      0.476      0.634      -0.301       0.493
home_away[T.H]       0.0078      0.033      0.238      0.812      -0.056       0.072
lagerror            -0.0607      0.033     -1.862      0.063      -0.125       0.003
shot_dist           -0.0181      0.003     -6.101      0.000      -0.024      -0.012
dribbles             0.0139      0.013      1.082      0.280      -0.011       0.039
touch_time          -0.0196      0.016     -1.239      0.216      -0.051       0.011
closest_def_dist     0.0238      0.006      3.712      0.000       0.011       0.036
==============================================================================
Omnibus:                     4325.364   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              113.453
Skew:                           0.064   Prob(JB):                     2.31e-25
Kurtosis:                       1.275   Cond. No.                         263.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Reg4_wls_player: Run weighted least squares regressions on individual players. Include the same set of control variables as in Reg3. The regression should be weighted by the number of shot per game (weight=1/shot_per_game).

In [45]:
formula = "error ~ lagerror + shot_dist + dribbles + touch_time + C(points) + C(quarter) + home_away + closest_def_dist"
results_4 = {}


for player in Shotlog_1415['shoot_player'].unique():
    player_data = Shotlog_1415[Shotlog_1415['shoot_player'] == player]

    weights = 1 / player_data['shot_per_game']

    reg4 = sm.wls(formula=formula, data=player_data, weights=weights).fit()
    results_4[player] = reg4

In [46]:
results_4["alonzo gee"].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  error   R-squared:                       0.239
Model:                            WLS   Adj. R-squared:                  0.146
Method:                 Least Squares   F-statistic:                     2.564
Date:                Sun, 10 Mar 2024   Prob (F-statistic):            0.00714
Time:                        06:02:41   Log-Likelihood:                -68.055
No. Observations:                 102   AIC:                             160.1
Df Residuals:                      90   BIC:                             191.6
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -0.1300      0.564     -0.231      0.818      -1.250       0.990
C(points)[T.3]       0.2975      0.258      1.154      0.251      -0.215       0.810
C(quarter)[T.2]      0.1240      0.549      0.226      0.822      -0.966       1.214
C(quarter)[T.3]      0.5462      0.546      1.000      0.320      -0.538       1.631
C(quarter)[T.4]      0.3730      0.537      0.695      0.489      -0.693       1.439
C(quarter)[T.5]      0.6464      0.686      0.943      0.348      -0.716       2.009
home_away[T.H]      -0.0071      0.095     -0.075      0.940      -0.195       0.181
lagerror            -0.3535      0.098     -3.608      0.001      -0.548      -0.159
shot_dist           -0.0322      0.013     -2.499      0.014      -0.058      -0.007
dribbles            -0.0664      0.063     -1.050      0.296      -0.192       0.059
touch_time           0.0368      0.056      0.662      0.510      -0.074       0.147
closest_def_dist     0.0206      0.014      1.424      0.158      -0.008       0.049
==============================================================================
Omnibus:                        6.362   Durbin-Watson:                   1.896
Prob(Omnibus):                  0.042   Jarque-Bera (JB):                2.872
Skew:                          -0.069   Prob(JB):                        0.238
Kurtosis:                       2.190   Cond. No.                         362.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [47]:
results_4["reggie jackson"].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  error   R-squared:                       0.083
Model:                            WLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     4.477
Date:                Sun, 10 Mar 2024   Prob (F-statistic):           6.58e-07
Time:                        06:02:52   Log-Likelihood:                -440.81
No. Observations:                 604   AIC:                             907.6
Df Residuals:                     591   BIC:                             964.9
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.1963      0.081      2.416      0.016       0.037       0.356
C(points)[T.3]      -0.0814      0.067     -1.216      0.224      -0.213       0.050
C(quarter)[T.2]     -0.0092      0.066     -0.139      0.889      -0.139       0.120
C(quarter)[T.3]      0.0007      0.070      0.010      0.992      -0.136       0.138
C(quarter)[T.4]     -0.0428      0.067     -0.641      0.521      -0.174       0.088
C(quarter)[T.5]     -0.1893      0.188     -1.007      0.314      -0.559       0.180
C(quarter)[T.6]     -0.4787      0.618     -0.775      0.439      -1.692       0.735
home_away[T.H]      -0.0003      0.040     -0.007      0.994      -0.079       0.079
lagerror             0.1290      0.040      3.254      0.001       0.051       0.207
shot_dist           -0.0135      0.004     -3.763      0.000      -0.021      -0.006
dribbles            -0.0042      0.015     -0.280      0.780      -0.034       0.025
touch_time          -0.0009      0.018     -0.050      0.960      -0.037       0.035
closest_def_dist     0.0186      0.010      1.806      0.071      -0.002       0.039
==============================================================================
Omnibus:                      694.278   Durbin-Watson:                   2.016
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               48.029
Skew:                           0.279   Prob(JB):                     3.72e-11
Kurtosis:                       1.736   Cond. No.                         590.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
results_4["stephen curry"].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  error   R-squared:                       0.078
Model:                            WLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     6.868
Date:                Sun, 10 Mar 2024   Prob (F-statistic):           3.42e-11
Time:                        06:02:56   Log-Likelihood:                -660.11
No. Observations:                 910   AIC:                             1344.
Df Residuals:                     898   BIC:                             1402.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.2125      0.057      3.738      0.000       0.101       0.324
C(points)[T.3]       0.1238      0.055      2.255      0.024       0.016       0.232
C(quarter)[T.2]     -0.0244      0.048     -0.507      0.612      -0.119       0.070
C(quarter)[T.3]     -0.0361      0.042     -0.866      0.387      -0.118       0.046
C(quarter)[T.4]     -0.0807      0.046     -1.746      0.081      -0.172       0.010
C(quarter)[T.5]      0.1106      0.207      0.535      0.593      -0.295       0.517
home_away[T.H]       0.0039      0.032      0.120      0.904      -0.060       0.068
lagerror            -0.0836      0.033     -2.557      0.011      -0.148      -0.019
shot_dist           -0.0181      0.003     -6.176      0.000      -0.024      -0.012
dribbles             0.0139      0.013      1.082      0.280      -0.011       0.039
touch_time          -0.0207      0.016     -1.331      0.183      -0.051       0.010
closest_def_dist     0.0234      0.006      3.748      0.000       0.011       0.036
==============================================================================
Omnibus:                     6350.130   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               88.184
Skew:                           0.089   Prob(JB):                     7.10e-20
Kurtosis:                       1.485   Cond. No.                         271.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [49]:
results_4["cole aldrich"].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  error   R-squared:                       0.162
Model:                            WLS   Adj. R-squared:                  0.109
Method:                 Least Squares   F-statistic:                     3.074
Date:                Sun, 10 Mar 2024   Prob (F-statistic):            0.00212
Time:                        06:02:59   Log-Likelihood:                -110.28
No. Observations:                 153   AIC:                             240.6
Df Residuals:                     143   BIC:                             270.9
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0614      0.170      0.360      0.719      -0.275       0.398
C(quarter)[T.2]     -0.0594      0.119     -0.499      0.619      -0.295       0.176
C(quarter)[T.3]      0.0504      0.119      0.425      0.672      -0.184       0.285
C(quarter)[T.4]      0.0230      0.121      0.190      0.850      -0.217       0.263
home_away[T.H]      -0.1658      0.079     -2.094      0.038      -0.322      -0.009
lagerror             0.0855      0.081      1.053      0.294      -0.075       0.246
shot_dist           -0.0355      0.015     -2.318      0.022      -0.066      -0.005
dribbles             0.0421      0.081      0.518      0.605      -0.118       0.203
touch_time          -0.0373      0.037     -1.003      0.317      -0.111       0.036
closest_def_dist     0.1139      0.034      3.314      0.001       0.046       0.182
==============================================================================
Omnibus:                       19.068   Durbin-Watson:                   1.961
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                6.895
Skew:                           0.236   Prob(JB):                       0.0318
Kurtosis:                       2.073   Cond. No.                         42.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""